# Chapter 12: Kubernetes Architecture

Having established secure artifact distribution through container registries, we now transition from single-host container management to distributed orchestration. Kubernetes (often abbreviated as K8s) is an open-source container orchestration platform that automates the deployment, scaling, and management of containerized applications. Originally designed by Google and now maintained by the Cloud Native Computing Foundation (CNCF), Kubernetes has become the de facto standard for container orchestration, supported by every major cloud provider and adopted by enterprises worldwide.

This chapter examines the architectural foundations of Kubernetes, exploring how the control plane maintains cluster state, how worker nodes execute workloads, and how these components communicate to form a resilient, scalable platform. Understanding these internals is essential for troubleshooting, security hardening, and optimizing production deployments.

## 12.1 Control Plane Components

The Kubernetes Control Plane functions as the brain of the cluster, making global decisions about the cluster state (such as scheduling workloads) and detecting and responding to cluster events (such as scaling when a deployment's replica count changes). Control plane components can run on any machine in the cluster, but for high availability and security, they typically reside on dedicated master nodes isolated from application workloads.

### Architectural Overview

A Kubernetes cluster consists of two primary segments:
- **Control Plane**: Manages cluster state and configuration
- **Data Plane (Worker Nodes)**: Executes containerized applications

The control plane exposes the Kubernetes API, which serves as the central nervous system through which all components—internal, user-facing, and automated systems—interact with the cluster.

### Core Control Plane Components

The control plane comprises several distinct processes, each with specific responsibilities:

**1. kube-apiserver**
The API server is the front end of the Kubernetes control plane. It exposes the Kubernetes REST API, handling all requests from users, automation, and internal components. It is the only component that communicates directly with the etcd datastore.

**2. etcd**
A consistent and highly-available distributed key-value store that serves as Kubernetes' backing store for all cluster data. It maintains the entire cluster state, including configuration data, operational metadata, and service discovery information.

**3. kube-scheduler**
Watches for newly created Pods with no assigned node and selects an optimal node for them to run on based on resource requirements, hardware constraints, affinity specifications, and other factors.

**4. kube-controller-manager**
Runs controller processes that regulate the state of the cluster. Controllers watch the shared state of the cluster through the API server and make changes attempting to move the current state toward the desired state.

**5. cloud-controller-manager** (optional)
Embeds cloud-specific control logic, allowing Kubernetes to interact with underlying cloud providers for load balancing, node management, and storage provisioning while maintaining vendor neutrality in core components.

### Control Plane Communication Patterns

All components communicate exclusively through the API server; they do not talk directly to each other. This hub-and-spoke model provides:
- **Auditability**: All state changes flow through a single chokepoint
- **Authorization**: Centralized access control enforcement
- **Validation**: Schema and policy verification at entry point

```yaml
# Example: Viewing control plane pods in a kubeadm cluster
apiVersion: v1
kind: Pod
metadata:
  name: kube-apiserver-master-1
  namespace: kube-system
spec:
  containers:
  - name: kube-apiserver
    image: registry.k8s.io/kube-apiserver:v1.28.0
    command:
      - kube-apiserver
      - --advertise-address=192.168.1.10
      - --allow-privileged=true
      - --authorization-mode=Node,RBAC
      - --client-ca-file=/etc/kubernetes/pki/ca.crt
      - --enable-admission-plugins=NodeRestriction
      - --etcd-servers=https://127.0.0.1:2379
      - --etcd-cafile=/etc/kubernetes/pki/etcd/ca.crt
      - --etcd-certfile=/etc/kubernetes/pki/apiserver-etcd-client.crt
      - --etcd-keyfile=/etc/kubernetes/pki/apiserver-etcd-client.key
      - --kubelet-client-certificate=/etc/kubernetes/pki/apiserver-kubelet-client.crt
      - --kubelet-client-key=/etc/kubernetes/pki/apiserver-kubelet-client.key
      - --kubelet-preferred-address-types=InternalIP,ExternalIP,Hostname
      - --proxy-client-cert-file=/etc/kubernetes/pki/front-proxy-client.crt
      - --proxy-client-key-file=/etc/kubernetes/pki/front-proxy-client.key
      - --requestheader-allowed-names=front-proxy-client
      - --requestheader-client-ca-file=/etc/kubernetes/pki/front-proxy-ca.crt
      - --requestheader-extra-headers-prefix=X-Remote-Extra-
      - --requestheader-group-headers=X-Remote-Group
      - --requestheader-username-headers=X-Remote-User
      - --secure-port=6443
      - --service-account-issuer=https://kubernetes.default.svc.cluster.local
      - --service-account-key-file=/etc/kubernetes/pki/sa.pub
      - --service-account-signing-key-file=/etc/kubernetes/pki/sa.key
      - --service-cluster-ip-range=10.96.0.0/12
      - --tls-cert-file=/etc/kubernetes/pki/apiserver.crt
      - --tls-private-key-file=/etc/kubernetes/pki/apiserver.key
```

## 12.2 Worker Nodes

While the control plane manages cluster state, worker nodes (also called minions or simply nodes) provide the compute capacity where containers actually run. Each worker node contains the necessary services to run Pods (the smallest deployable units in Kubernetes) and is managed by the control plane.

### Node Components

Every worker node runs three essential processes:

**1. kubelet**
An agent that runs on each node in the cluster. It ensures that containers are running in a Pod. The kubelet takes a set of PodSpecs provided through various mechanisms (primarily the API server) and ensures that the containers described in those PodSpecs are running and healthy.

**2. kube-proxy**
A network proxy that maintains network rules on nodes, enabling the Kubernetes Service abstraction. It handles network communications inside or outside the cluster, performing connection forwarding or load balancing across Pods.

**3. Container Runtime**
The software responsible for running containers. Kubernetes supports multiple container runtimes through the Container Runtime Interface (CRI):
- **containerd**: The industry-standard runtime (default in most modern distributions)
- **CRI-O**: Lightweight alternative optimized for Kubernetes
- **Docker Engine**: Supported via cri-dockerd adapter (deprecated as direct runtime)

### Node Architecture Deep Dive

Worker nodes require specific configuration for optimal operation:

**Systemd Services:**
Modern Kubernetes distributions use systemd to manage node components:

```ini
# /etc/systemd/system/kubelet.service
[Unit]
Description=Kubernetes Kubelet
Documentation=https://kubernetes.io/docs/reference/command-line-tools-reference/kubelet/
After=containerd.service
Requires=containerd.service

[Service]
ExecStart=/usr/local/bin/kubelet \
  --bootstrap-kubeconfig=/etc/kubernetes/bootstrap-kubelet.conf \
  --kubeconfig=/etc/kubernetes/kubelet.conf \
  --config=/var/lib/kubelet/config.yaml \
  --container-runtime-endpoint=unix:///run/containerd/containerd.sock \
  --pod-infra-container-image=registry.k8s.io/pause:3.9
Restart=always
RestartSec=5

[Install]
WantedBy=multi-user.target
```

**Node Resource Allocation:**
Nodes reserve resources for system daemons and Kubernetes components:

```yaml
# /var/lib/kubelet/config.yaml
apiVersion: kubelet.config.k8s.io/v1beta1
kind: KubeletConfiguration
systemReserved:
  cpu: "500m"
  memory: "512Mi"
  ephemeral-storage: "1Gi"
kubeReserved:
  cpu: "500m"
  memory: "1Gi"
  ephemeral-storage: "1Gi"
evictionHard:
  memory.available: "500Mi"
  nodefs.available: "10%"
evictionSoft:
  memory.available: "1Gi"
  nodefs.available: "15%"
evictionSoftGracePeriod:
  memory.available: "1m"
  nodefs.available: "1m"
```

## 12.3 Kubernetes Objects Overview

Kubernetes objects are persistent entities in the Kubernetes system that represent the state of your cluster. They describe what containerized applications are running, the resources available to them, and policies around their behavior (such as restart policies, upgrades, and fault-tolerance).

### Object Specification Pattern

Every Kubernetes object follows a consistent structure:

```yaml
apiVersion: v1          # Version of the Kubernetes API
kind: Pod               # Type of object
metadata:               # Identifying information
  name: example-pod
  namespace: default
  labels:
    app: web
    tier: frontend
spec:                   # Desired state description
  containers:
  - name: nginx
    image: nginx:1.25
    ports:
    - containerPort: 80
status:                 # Current state (populated by system)
  phase: Running
  conditions:
  - type: Ready
    status: "True"
```

### Fundamental Object Categories

Kubernetes objects organize into logical groups:

**Workload Objects**: Manage container deployment and execution
- Pods, Deployments, StatefulSets, DaemonSets, Jobs, CronJobs

**Service Objects**: Enable network access and discovery
- Services, Ingress, NetworkPolicies

**Config/Storage Objects**: Manage configuration and persistence
- ConfigMaps, Secrets, PersistentVolumes, PersistentVolumeClaims, StorageClasses

**Cluster Objects**: Define cluster-wide behavior
- Nodes, Namespaces, ResourceQuotas, LimitRanges, ServiceAccounts, Roles, RoleBindings

### Declarative Management Philosophy

Kubernetes operates on a declarative model: you specify the desired state, and the control plane works continuously to maintain that state. This differs from imperative systems where you issue specific commands to create or modify resources.

**Imperative approach (avoid in production):**
```bash
kubectl run nginx --image=nginx
kubectl scale deployment nginx --replicas=3
kubectl expose deployment nginx --port=80
```

**Declarative approach (recommended):**
```bash
# Apply configuration file describing desired state
kubectl apply -f nginx-deployment.yaml

# Version control your infrastructure
git add nginx-deployment.yaml
git commit -m "Add nginx deployment with 3 replicas"
git push
```

The declarative model enables GitOps workflows, where the git repository serves as the single source of truth for infrastructure state.

## 12.4 API Server and etcd

The API server and etcd form the heart of the Kubernetes control plane, handling all state persistence and serving as the communication hub for the entire cluster.

### API Server Deep Dive

The Kubernetes API server (kube-apiserver) validates and configures data for API objects, serving as the front end to the cluster's shared state. It follows RESTful principles and uses JSON or Protocol Buffers for serialization.

**Request Flow:**
1. **Authentication**: Verifies identity (X.509 certificates, Bearer tokens, OIDC, webhook)
2. **Authorization**: Determines permissions (RBAC, ABAC, Node authorization, Webhook)
3. **Admission Control**: Modifies or validates requests (mutating/validating webhooks, built-in plugins)
4. **ETCD Interaction**: Reads/writes state to backing store
5. **Response**: Returns result to client

**API Groups and Versions:**
Kubernetes APIs organize into groups and versions to enable evolution:

```
/apis/apps/v1/deployments       # Stable apps API
/apis/apps/v1beta1/deployments  # Deprecated beta version
/api/v1/pods                    # Core API (legacy group)
/apis/networking.k8s.io/v1/ingresses  # Networking API
```

**Aggregation Layer:**
The API server supports extension through API Aggregation, allowing custom APIs to register alongside native ones:

```yaml
apiVersion: apiregistration.k8s.io/v1
kind: APIService
metadata:
  name: v1beta1.metrics.k8s.io
spec:
  service:
    name: metrics-server
    namespace: kube-system
  group: metrics.k8s.io
  version: v1beta1
  insecureSkipTLSVerify: true
  groupPriorityMinimum: 100
  versionPriority: 100
```

### etcd: The Cluster Brain

etcd is a distributed, reliable key-value store written in Go that uses the Raft consensus algorithm to manage a highly-available replicated log. It is the ultimate source of truth for Kubernetes cluster state.

**Data Organization:**
etcd stores all Kubernetes data under the `/registry` prefix:

```
/registry/pods/default/nginx-7d8bc9c5b8-abc12
/registry/deployments/default/nginx-deployment
/registry/secrets/default/app-token
/registry/nodes/worker-node-1
```

**Consensus and Replication:**
etcd operates typically as a cluster of 3, 5, or 7 members (odd numbers prevent split-brain scenarios). The Raft algorithm ensures:
- **Leader Election**: One node acts as leader handling all writes
- **Log Replication**: Leader replicates entries to followers
- **Safety**: Committed entries persist even if minority of nodes fail

**Performance Characteristics:**
- Optimized for read-heavy workloads (Kubernetes is read-heavy)
- Writes require consensus (latency increases with distance between nodes)
- Recommended storage: SSD/NVMe for write-ahead logs
- Defragmentation required periodically to reclaim space

**Backing Up etcd:**
Regular backups are critical for disaster recovery:

```bash
# Snapshot etcd (run on master node or via pod)
ETCDCTL_API=3 etcdctl snapshot save /backup/etcd-snapshot-$(date +%Y%m%d).db \
  --endpoints=https://127.0.0.1:2379 \
  --cacert=/etc/kubernetes/pki/etcd/ca.crt \
  --cert=/etc/kubernetes/pki/etcd/server.crt \
  --key=/etc/kubernetes/pki/etcd/server.key

# Verify snapshot
ETCDCTL_API=3 etcdctl snapshot status /backup/etcd-snapshot-20231102.db

# Restore (disaster recovery)
ETCDCTL_API=3 etcdctl snapshot restore snapshot.db \
  --data-dir=/var/lib/etcd-new \
  --initial-cluster-token=etcd-cluster-1 \
  --initial-cluster=master-1=https://192.168.1.10:2380 \
  --initial-advertise-peer-urls=https://192.168.1.10:2380
```

**Security Considerations:**
- Enable TLS encryption for peer and client communication
- Use dedicated CA for etcd PKI separate from Kubernetes CA
- Enable authentication (username/password or certificates)
- Enable audit logging for compliance requirements

## 12.5 Scheduler and Controller Manager

While the API server and etcd store state, the scheduler and controller manager actively work to reconcile desired state with actual cluster conditions.

### kube-scheduler

The scheduler watches for newly created Pods with no assigned node (`nodeName` field empty) and selects the best node for them based on scheduling principles.

**Scheduling Algorithm:**
1. **Filtering (Predicates)**: Remove nodes that cannot run the Pod
   - Resource requirements (CPU, memory, GPU)
   - Node selectors/affinity rules
   - Taints and tolerations
   - Volume availability
   - Inter-pod affinity/anti-affinity

2. **Scoring (Priorities)**: Rank remaining nodes by suitability
   - Least requested resources (spread load)
   - Node affinity preferences
   - Pod topology spread

3. **Binding**: Assign Pod to highest-scoring node

**Scheduling Constraints Example:**
```yaml
apiVersion: v1
kind: Pod
spec:
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpressions:
          - key: kubernetes.io/arch
            operator: In
            values: ["amd64"]
    podAntiAffinity:
      preferredDuringSchedulingIgnoredDuringExecution:
      - weight: 100
        podAffinityTerm:
          labelSelector:
            matchExpressions:
            - key: app
              operator: In
              values: ["web"]
          topologyKey: kubernetes.io/hostname
  tolerations:
  - key: "dedicated"
    operator: "Equal"
    value: "web"
    effect: "NoSchedule"
```

**Custom Scheduling:**
For advanced use cases, you can deploy multiple schedulers:
- Default scheduler for general workloads
- GPU scheduler for ML training jobs
- Bin-packing scheduler for cost optimization

```yaml
apiVersion: v1
kind: Pod
spec:
  schedulerName: gpu-scheduler  # Use custom scheduler
  containers:
  - name: training
    image: tensorflow/tensorflow:latest-gpu
    resources:
      limits:
        nvidia.com/gpu: 1
```

### kube-controller-manager

Controllers are control loops that watch the shared state of the cluster through the API server and make changes attempting to move the current state toward the desired state.

**Built-in Controllers:**

| Controller | Responsibility |
|------------|----------------|
| **Node Controller** | Monitors node health, evicts Pods from unreachable nodes after timeout (default 5m) |
| **Replication Controller** | Maintains correct Pod count for ReplicaSets |
| **Endpoints Controller** | Populates Endpoints objects (joins Services & Pods) |
| **Service Account & Token Controllers** | Create default accounts and API access tokens for new namespaces |
| **Namespace Controller** | Handles namespace deletion, cleaning up resources |
| **Deployment Controller** | Manages rolling updates and rollbacks |
| **StatefulSet Controller** | Manages ordered deployment and persistence |
| **Job Controller** | Watches for Job objects and creates Pods to complete work |

**Controller Pattern Implementation:**
All controllers follow the same fundamental pattern:
1. Read resource state via API server watch
2. Compare desired state with observed state
3. Execute actions to converge states
4. Repeat indefinitely (level-triggered, not edge-triggered)

**Cloud Controller Manager:**
Separates cloud-specific logic from core Kubernetes:
- **Node Controller**: Cloud instance verification
- **Route Controller**: Cloud route table configuration
- **Service Controller**: Cloud load balancer provisioning

## 12.6 Kubelet and Kube-proxy

These node agents translate control plane directives into container runtime operations and network configurations.

### Kubelet: The Node Agent

The kubelet is the primary "node agent" that runs on each node. It registers the node with the API server using one of:
- Hostname (default)
- Override flag (`--hostname-override`)
- Cloud provider-specified name

**Pod Lifecycle Management:**
The kubelet receives PodSpecs primarily from:
1. API server (primary source)
2. Local files (static Pods)
3. HTTP endpoints (less common)

It ensures containers described in PodSpecs are running and healthy via:
- **Container Runtime Interface (CRI)**: Calls to containerd/CRI-O to create/start/stop containers
- **Readiness/Liveness Probes**: HTTP/TCP/exec checks to determine container health
- **Resource Monitoring**: cAdvisor integration for container metrics

**Static Pods:**
Managed directly by kubelet without API server oversight (used for control plane bootstrapping):

```yaml
# /etc/kubernetes/manifests/etcd.yaml on master node
apiVersion: v1
kind: Pod
metadata:
  name: etcd
  namespace: kube-system
spec:
  containers:
  - name: etcd
    image: registry.k8s.io/etcd:3.5.9-0
    command:
    - etcd
    - --advertise-client-urls=https://192.168.1.10:2379
    - --cert-file=/etc/kubernetes/pki/etcd/server.crt
    # ... additional args
  volumes:
  - name: etcd-certs
    hostPath:
      path: /etc/kubernetes/pki/etcd
      type: Directory
```

**Node Status Conditions:**
Kubelet reports node status including:
- **Ready**: Node healthy and accepting Pods
- **DiskPressure**: Available disk space low
- **MemoryPressure**: Available memory low
- **PIDPressure**: Too many processes
- **NetworkUnavailable**: Network not configured correctly

### Kube-proxy: Network Proxy

Kube-proxy maintains network rules on nodes, enabling the Kubernetes Service abstraction. It implements Services using:
- **iptables** (default, mature)
- **IPVS** (for large clusters, better performance)
- **Userspace** (legacy, slow)

**Service Implementation:**
When a Service is created, kube-proxy on every node updates local networking:

```bash
# View iptables rules created by kube-proxy
iptables -t nat -L KUBE-SERVICES -n | grep http

# Chain KUBE-SVC-XXXXXXXXX (1 references)
# target     prot opt source               destination
# KUBE-SEP-XXXXXXXXX  all  --  0.0.0.0/0            10.96.123.45         /* default/http: cluster IP */ tcp dpt:80
```

**Modes of Operation:**

**iptables mode** (default):
- Adds rules to iptables for each Service endpoint
- O(n) complexity where n = number of Services
- Suitable for clusters with < 1000 Services

**IPVS mode** (recommended for large scale):
- Uses Linux Virtual Server kernel module
- O(1) complexity regardless of Service count
- Supports multiple load balancing algorithms (rr, lc, dh, sh, sed, nq)

```yaml
# kube-proxy ConfigMap
apiVersion: kubelet.config.k8s.io/v1alpha1
kind: KubeProxyConfiguration
mode: "ipvs"
ipvs:
  scheduler: "rr"  # Round-robin
  minSyncPeriod: 0s
  syncPeriod: 30s
```

**Kernel Space vs Userspace:**
Modern kube-proxy operates in kernel space (iptables/IPVS) for performance, avoiding the overhead of copying packets through user space that characterized early Kubernetes implementations.

## 12.7 Cluster Communication

Understanding how cluster components communicate is essential for troubleshooting network issues and securing cluster traffic.

### Communication Paths

**1. Node to Control Plane:**
- Kubelet → API Server (HTTPS, port 6443)
- Uses certificates for authentication
- Watch connections maintained for real-time updates

**2. Control Plane to Node:**
- API Server → Kubelet (HTTPS, port 10250)
  - Used for `kubectl logs`, `kubectl exec`, `kubectl attach`
  - Requires proper certificate authentication
- API Server → Node, Pod, Service (via kube-proxy)

**3. Intra-Pod Communication:**
- Containers within a Pod share network namespace (localhost)
- Communicate via `localhost:port`
- Share IP address and port space

**4. Pod to Pod Communication:**
- All Pods can communicate without NAT
- Each Pod has unique IP in cluster CIDR
- Implemented by CNI plugins (Calico, Cilium, Flannel, Weave)

**5. Pod to Service Communication:**
- Virtual IPs (ClusterIPs) abstract Pod endpoints
- Kube-proxy load balances across healthy Pods
- DNS resolves Service names to ClusterIPs

### Network Requirements

Kubernetes imposes fundamental network requirements (the "Kubernetes Network Model"):

1. **All containers can communicate with all other containers without NAT**
2. **All nodes can communicate with all containers (and vice versa) without NAT**
3. **The IP that a container sees itself as is the same IP that others see it as**

**CIDR Ranges:**
Distinct non-overlapping ranges required:
- **Pod CIDR**: IP range assigned to Pods (e.g., 10.244.0.0/16)
- **Service CIDR**: Virtual IP range for Services (e.g., 10.96.0.0/12)
- **Node CIDR**: Physical node IP addresses

### TLS and Certificate Architecture

Kubernetes uses a PKI infrastructure for component authentication:

**Certificate Types:**
- **Client certificates**: For users (admin, developer) and components (kubelet, scheduler)
- **Server certificates**: For API server, etcd, kubelet serving
- **CA certificates**: Root of trust for cluster

**Certificate Locations (standard kubeadm):**
```
/etc/kubernetes/pki/
├── ca.crt              # Cluster CA
├── ca.key              # Cluster CA key (keep secure/offline)
├── apiserver.crt       # API server serving cert
├── apiserver-kubelet-client.crt  # API server client to kubelet
├── etcd/
│   ├── ca.crt          # etcd CA (separate for isolation)
│   ├── server.crt      # etcd serving cert
│   └── peer.crt        # etcd peer communication
└── front-proxy-ca.crt  # Aggregated API server CA
```

**Certificate Rotation:**
Short-lived certificates improve security:
- Kubeadm automatically rotates certificates (1 year default)
- Can configure external issuers (Vault, cert-manager) for automated rotation

## 12.8 High Availability Architecture

Production Kubernetes clusters must survive control plane node failures without service interruption.

### Control Plane HA Topologies

**Stacked etcd Topology (simpler, less hardware):**
- Control plane components and etcd share nodes
- Minimum 3 master nodes for quorum
- If node fails, loses both API server and etcd member

**External etcd Topology (more complex, higher availability):**
- etcd runs on separate nodes from API servers
- API servers can be horizontally scaled independently
- Survives loss of API server nodes without affecting etcd quorum

### HA Implementation

**Load Balancing API Servers:**
External load balancer distributes traffic across API server replicas:

```yaml
# HAProxy configuration example
global
    daemon
    maxconn 4096

defaults
    mode http
    timeout connect 5s
    timeout client 30s
    timeout server 30s

frontend kubernetes-apiserver
    bind *:6443
    default_backend kubernetes-apiserver

backend kubernetes-apiserver
    balance roundrobin
    server master1 192.168.1.10:6443 check
    server master2 192.168.1.11:6443 check
    server master3 192.168.1.12:6443 check
```

**etcd Clustering:**
Odd number of members (3, 5, 7) to maintain quorum:
- 3 nodes: tolerate 1 failure
- 5 nodes: tolerate 2 failures
- 7 nodes: tolerate 3 failures

Adding more than 7 etcd members typically reduces performance due to consensus overhead.

**Leader Election:**
Components like scheduler and controller-manager use leader election to ensure only one instance is active at a time, preventing conflicts:

```bash
# View leader election leases
kubectl get leases -n kube-system
# NAME                      HOLDER                                                                           AGE
# kube-controller-manager   master-1_12345...                                                                24h
# kube-scheduler            master-2_67890...                                                                24h
```

### Disaster Recovery Planning

**Backup Strategy:**
1. **etcd snapshots**: Cluster state (critical)
2. **Persistent volume backups**: Application data
3. **Configuration backups**: YAML manifests, certificates
4. **Registry replication**: Container images

**Recovery Procedures:**
Document and test recovery of:
- Single etcd member replacement
- Complete etcd cluster restoration
- API server certificate regeneration
- Worker node replacement

---

## Chapter Summary and Preview

In this chapter, we established the architectural foundations of Kubernetes, dissecting the control plane components (API server, etcd, scheduler, controller manager) that maintain cluster state, and the worker node components (kubelet, kube-proxy, container runtime) that execute workloads. We explored the declarative object model, the critical role of etcd as the distributed backing store using Raft consensus, and the network communication patterns binding the cluster together. High availability strategies ensure production resilience through stacked or external etcd topologies, load-balanced API servers, and automated leader election.

**Key Takeaways:**
- The API server is the central hub; all components communicate through it, enabling centralized audit and authorization.
- etcd requires odd-numbered clusters (3, 5, 7) for quorum and serves as the ultimate source of truth requiring regular backups.
- Contators operate as reconciliation loops continuously moving observed state toward desired state.
- Kubelet translates PodSpecs into container runtime operations while kube-proxy implements Service networking via iptables or IPVS.
- Kubernetes mandates a flat network model where all Pods communicate without NAT, implemented by CNI plugins.
- Production clusters require HA control planes with redundant API servers and etcd members behind load balancers.

**Next Chapter Preview:**
Chapter 13: Getting Started with Kubernetes transitions from architecture theory to practical deployment. You will install local Kubernetes clusters using Minikube, Kind, or k3d; configure kubectl command-line access; create your first Namespace; and deploy initial Pods. This hands-on chapter bridges the architectural understanding gained here with the operational skills needed to manage real workloads, preparing you for Chapter 14's deep dive into core Kubernetes resources like Deployments, Services, and ConfigMaps.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../2. docker_fundamentals/11. docker_registries.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='13. getting_started_with_kubernetes.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
